In [169]:
from USTs import USTs
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from scipy.optimize import newton
import pandas_market_calendars as mcal
import pandas as pd
import numpy as np

In [170]:
# Bill pricing
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)

bill_1 = USTs(auction_data= None, price_data=None).get_bill_discount_rate(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_1
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2
issue_date = datetime.date(1990, 6, 7)
maturity_date = datetime.date(1991, 6, 6)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=92.265000,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

8.237

In [ ]:
holiday_array = mcal.get_calendar('NYSE').holidays().holidays
start, end = datetime.date(1990, 1, 1), datetime.date(2060, 1, 1)
holidays_list = [pd.to_datetime(np_date).date() for np_date in holiday_array]
holidays = [holiday for holiday in holidays_list if start < holiday < end]
nyse_calendar = mcal.get_calendar('NYSE').regular_holidays

def adjust_for_bad_day(date: datetime.date) -> datetime.date: # type: ignore
    while date.weekday() in [5, 6] or date in holidays:
        date = date + timedelta(days=1)
    return date

def get_dates_and_cashflows(issue_date: datetime.date,
                            maturity_date: datetime.date,
                            coupon: float,
                            FV: int = 100):
    payment_dates = [maturity_date]
    current_date = maturity_date
    while current_date > issue_date:
        current_date = maturity_date - relativedelta(months=len(payment_dates) * 6)
        if current_date > issue_date:
            payment_dates.append(current_date)
    payment_dates = sorted(payment_dates)
    
    coupon_amt = coupon / 2
    return_list = [(issue_date, 0.0)] # Issue date
    for date in payment_dates[:-1]:
        return_list.append((date, coupon_amt))
    return_list.append((payment_dates[-1], coupon_amt + FV)) # Maturity with principal repayment
    return return_list

def calculate_accrued_interest(dates_and_cashflows, as_of_date) -> float:
    for date, _ in dates_and_cashflows:
        if date > as_of_date:
            next_coupon_date = date
            previous_index = dates_and_cashflows.index((date, _)) - 1
            last_date = dates_and_cashflows[previous_index][0]
            break

    days_in_period = (next_coupon_date - last_date).days
    days_accrued = (as_of_date - last_date).days
    if days_in_period == 0:
        return 0.0

    coupon_amt = dates_and_cashflows[1][1]
    accrued = coupon_amt * days_accrued / days_in_period
    return accrued

def calculate_bond_price(issue_date: datetime.date,
                         maturity_date: datetime.date,
                         as_of_date: datetime.date,
                         coupon: float,
                         discount_rate: float,
                         dirty: bool = False) -> float:
    
    dates_and_cashflows = get_dates_and_cashflows(issue_date=issue_date,
                                                      maturity_date=maturity_date,
                                                      coupon=coupon)
    for date, _ in dates_and_cashflows:
        if date > as_of_date:
            first_pmt_date = adjust_for_bad_day(date)
            previous_index = dates_and_cashflows.index((date, _)) - 1
            last_date = adjust_for_bad_day(dates_and_cashflows[previous_index][0])
            first_pmt_period = (first_pmt_date - last_date).days
            time_to_pmt = (first_pmt_date - as_of_date).days
            break
    first_period_fraction = time_to_pmt / first_pmt_period

    PV = 0
    exponent = first_period_fraction
    previous_date = None
    for unadjusted_date, cashflow in dates_and_cashflows:
        date = adjust_for_bad_day(unadjusted_date)
        if date > as_of_date:
            if previous_date:
                days_in_period = (date - previous_date).days
                exponent += (days_in_period / (365 / 2))
            pmt_pv = cashflow / ((1 + discount_rate/2) ** exponent)
            PV += pmt_pv
            previous_date = date

    if not dirty: # Clean price
        accrued = calculate_accrued_interest(dates_and_cashflows=dates_and_cashflows,
                                             as_of_date=as_of_date)
        PV -= accrued
    return PV

def error_function(discount_rate: float,
                   price: float,
                   issue_date: datetime.date,
                   maturity_date: datetime.date,
                   as_of_date: datetime.date,
                   coupon: float,
                   dirty: bool=False):
    calculated_price = calculate_bond_price(issue_date=issue_date,
                                            maturity_date=maturity_date,
                                            discount_rate=discount_rate,
                                            coupon=coupon,
                                            as_of_date=as_of_date,
                                            dirty=dirty)
    error = price - calculated_price
    return error


def get_coupon_ytm(price: float, 
            issue_date: datetime.date,
            maturity_date: datetime.date,
            as_of_date: datetime.date,
            coupon: float,
            dirty: bool=False):
    guess = coupon / 100

    try:
        ytm = newton(
            func=error_function,
            x0=guess,
            args=(price, issue_date, maturity_date, as_of_date, coupon, dirty)
        )
        return round(float(ytm * 100), 6)
    except RuntimeError:
        return None

In [172]:
PRICE = 100.32670 - 0.100138
ISSUE = datetime.datetime(2001, 8, 31)
MATURITY = datetime.datetime(2003, 8, 31)
RATE = 3.625
SETTLEMENT = datetime.datetime(2001, 9, 10)

In [181]:
get_dates_and_cashflows(ISSUE.date(), MATURITY.date(), RATE)

[(datetime.date(2001, 8, 31), 0.0),
 (datetime.date(2002, 2, 28), 1.8125),
 (datetime.date(2002, 8, 31), 1.8125),
 (datetime.date(2003, 2, 28), 1.8125),
 (datetime.date(2003, 8, 31), 101.8125)]

In [173]:
get_coupon_ytm(price=PRICE,
        issue_date=ISSUE.date(),
        maturity_date=MATURITY.date(),
        as_of_date=SETTLEMENT.date(),
        coupon=RATE)

3.487633

In [174]:
import rateslib as rl
bond = rl.FixedRateBond(
    effective=ISSUE,
    termination=MATURITY,
    fixed_rate=RATE,
    spec="us_gb"  # US Government Bond
)
bond.ytm(price=PRICE, settlement=SETTLEMENT, dirty=False)

3.5046797329273054

In [175]:
bond.accrued(settlement=SETTLEMENT)

0.10013812154696132

In [176]:
USTs(auction_data=None,price_data=None).get_coupon_ytm(price=PRICE, issue_date=ISSUE.date(),
                    maturity_date=MATURITY.date(),
                    as_of_date=SETTLEMENT.date(),
                    coupon=RATE)

0.10013812154696132
0.10013812154696132
0.10013812154696132
0.10013812154696132
0.10013812154696132
0.10013812154696132


3.506683

In [177]:
(datetime.date(2002, 2, 28) - datetime.date(2001, 9, 10)).days # 171 days
# (datetime.date(2001, 7, 31) - datetime.date(2001, 1, 31)).days # 181 days

# First period is 171 / 181.
# Approximation is to use this for first period, discounting by YTM rate, and add 1 to this
# fraction for each subsequent period. However, this does not lead to an exact result.

# Accounting for 'bad days' - Aug 31 2003 3 5/8ths - four coupon periods.
# Accurate calculation method:
    # First coupon is as above, 



171

In [178]:
(adjust_for_bad_day(datetime.date(2002, 2, 28)) - adjust_for_bad_day(datetime.date(2001, 8, 31))).days 
# 181 days (171 days remaining) - 171 / 181 -> 0.9447513812154696
# 187 days - second CF - 187 / (365 / 2) -> 1.0246575342465754
# 178 days - third CF - 178 / (365 / 2) -> 0.9753424657534246
# 186 days - final CF - 186 / (365/2) -> 1.0191780821917809

181

In [179]:
(datetime.date(2001, 9, 10) - datetime.date(2001, 8, 31)).days

10

In [180]:
10 * 3.625 / 2 /184

0.09850543478260869